# Mastering Applied Skills in Management, Analytics and Entrepreneurship I

## DATA COLLECTION TECHNIQUES
## Part V. Web scraping with Selenium

JupyterHub installation includes [Selenium with Python](https://selenium-python.readthedocs.io/) which provides a simple API to write functional/acceptance tests using Selenium WebDriver or just to scrap sites over the Internet.

### 1. Selenium library

In [ ]:
import io
import time
import matplotlib.pyplot as plt
import selenium
from selenium import webdriver
from selenium.webdriver import FirefoxOptions

First step is to create browser to access the site, this browser will be our eyes and hands for this task. For the site this browser will look like any human-like user.

In [ ]:
opts = FirefoxOptions()
opts.add_argument('--headless')
browser = webdriver.Firefox(options=opts)

In [ ]:
browser

In [ ]:
browser.name

In [ ]:
browser.current_url

### 2. Basic demo

We will take a task from the last year diploma project which was about AI job description analysis. First step will be to collect data from the site.

In [ ]:
url_ai_jobs = 'https://ai-jobs.net'
print(url_ai_jobs)

In [ ]:
# put the url to our browser
browser.get(url_ai_jobs)

In [ ]:
# now our browser got the url
browser.current_url

In [ ]:
img_bytes = browser.get_full_page_screenshot_as_png()

In [ ]:
from PIL import Image

# we need some transformation 
# because image is in bytes
plt.figure(figsize=(16, 64))
img = Image.open(io.BytesIO(img_bytes))
plt.imshow(img)
plt.show()

In [ ]:
# easy task, just collect text from the page
text_from_site = browser.find_element('xpath', 'html').text

In [ ]:
print(text_from_site)

In [ ]:
# remember this number!
len(text_from_site)

### 3. Click buttons

What is the problem with `AI jobs` site? Why can not we use `BeautifulSoup` library as usual?

Because of the `Load more jobs` button. We need new page to open, but soup from `BeautifulSoup` collect data but can not click `Load more` buttons.

In [ ]:
# find the button first
# `Developer mode` will help us again
# click `Explore element` 
# and `Copy XPath`
# result is `//*[@id="load-more-jobs"]/a`

browser.find_element('xpath', './/*[@id="load-more-jobs"]/a/h5')

In [ ]:
browser.find_element('xpath', './/*[@id="load-more-jobs"]/a/h5').text

In [ ]:
# try to click it

browser.find_element('xpath', './/*[@id="load-more-jobs"]/a/h5').click()

In [ ]:
# below is an ugly workaround
# but it works!
# if you want a production solution
# see here https://stackoverflow.com/questions/56085152
# /selenium-python-error-element-could-not-be-scrolled-into-view

import time

flag = True

while flag:
    try:
        browser.find_element('xpath', './/*[@id="load-more-jobs"]/a/h5').click()
        flag = False
    except:
        print('sleep 1 sec and then click again')
        time.sleep(1)

In [ ]:
text_from_site = browser.find_element('xpath', 'html').text

In [ ]:
# look at the number carefully
len(text_from_site)

### 4. Click buttons wisely

#### 4.1. Make many clicks

In [ ]:
counter = 0
while counter < 3: # or 'while True:' for endless
    try:
        # click button
        browser.find_element('xpath', './/*[@id="load-more-jobs"]/a/h5').click()
        # ...and then collect data from site
        text_from_site = browser.find_element('xpath', 'html').text
        counter += 1
        print('click', counter, '| text', len(text_from_site))
    except Exception as e:
        print('sleep 1 sec and then click again')
        time.sleep(1)
text_from_site = browser.find_element('xpath', 'html').text
len(text_from_site)

#### 4.2. Try to find out what we have collected

It is a good idea to search elements with help of `XPath` and we can easily get it with help of `Developer mode`.

In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
# first copy XPath `//*[@id="job-list"]/li[1]/div/a`
# but here is a trick - we need all elements
# so the right XPath will be `//*[@id="job-list"]/li/div/a` 

jobs = browser.find_elements(By.XPATH, '//*[@id="job-list"]/li/div/a')
len(jobs)

In [ ]:
one_job = jobs[1]
one_job

In [ ]:
# data we can extract
one_job.text

In [ ]:
# we can extract sub-elements with `get_attribute`
# and `Developer mode` for copying the structure
# of the desired element e.g.
# <a class="col pt-2 pb-3" href="/job/73954-data-engineer/" title="View details for this job">
# ...
# </a>

one_job.get_attribute(name='href')

In [ ]:
one_job.get_attribute(name='title')

In [ ]:
# we can go deeper again with help of XPath
# for sub-elements but we need to edit a path
# from `//*[@id="job-list"]/li[1]/div/a/div/span[2]`
# to `.//div/span[2]` because `//*[@id="job-list"]/li/div/a`
# refers to whole element of job description
one_job.find_element(By.XPATH, './/div/span[2]').text

In [ ]:
one_job.find_element(By.XPATH, './/div/span[3]').text

In [ ]:
one_job.find_element(By.XPATH, './/div/div[1]').text

In [ ]:
one_job.find_element(By.XPATH, './/div/div[2]').text

In [ ]:
one_job.find_element(By.XPATH, './/h3').text

In [ ]:
one_job.find_element(By.XPATH, './/p').text

In [ ]:
# some trick is required for many elements in list
# look for `<span class="badge rounded-pill text-bg-light">`
[x.text for x in one_job.find_elements(By.XPATH, ".//span[@class='badge rounded-pill text-bg-light']")]

In [ ]:
[x.text for x in one_job.find_elements(By.XPATH, ".//span[@class='badge rounded-pill text-bg-success']")]

#### 4.3. Make a loop for all job descriptions

In [ ]:
from tqdm.auto import tqdm

In [ ]:
all_jobs = []
for job in tqdm(jobs):
    job_dict = {}
    job_dict['url'] = job.get_attribute(name='href')
    job_dict['location'] = job.find_element(By.XPATH, './/div/span[2]').text
    job_dict['type'] = job.find_element(By.XPATH, './/div/span[3]').text
    job_dict['level'] = job.find_element(By.XPATH, './/div/div[1]').text
    try:
        job_dict['salary_range'] = job.find_element(By.XPATH, './/div/div[2]').text
    except:
        job_dict['salary_range'] = ''
    job_dict['position'] = job.find_element(By.XPATH, './/h3').text
    job_dict['company'] = job.find_element(By.XPATH, './/p').text
    job_dict['skills'] = [
        x.text 
        for x in job.find_elements(By.XPATH, ".//span[@class='badge rounded-pill text-bg-light']")
    ]
    job_dict['benefits'] = [
        x.text 
        for x in job.find_elements(By.XPATH, ".//span[@class='badge rounded-pill text-bg-success']")
    ]
    all_jobs.append(job_dict)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(all_jobs)

In [ ]:
df

### 5. Single position with Selenium

In [ ]:
print(all_jobs[0]['url'])

In [ ]:
df.url[0]

We need new browser for the new url:

In [ ]:
opts = FirefoxOptions()
opts.add_argument('--headless')
browser = webdriver.Firefox(options=opts)

In [ ]:
url_ai_job = all_jobs[0]['url']
browser.get(url_ai_job)

In [ ]:
browser.find_element('xpath', 'html').text

In [ ]:
text = browser.find_element(By.XPATH, '//*[@id="content"]/section/div/div/script')

In [ ]:
text.get_attribute('innerHTML')

In [ ]:
import json

In [ ]:
data = json.loads(text.get_attribute('innerHTML'))
data

In [ ]:
data['baseSalary']

## LAB WORK #4

Collect at least 500 jobs from [ai-jobs.net](https://ai-jobs.net) and find maximum (maxValue) base salary in USD per year all across over the job descriptions collected.

### <font color='green'>Solution</font>

In [ ]:
# YOUR CODE HERE